In [1]:
!pip install torch torchvision torchaudio
!pip install ogb
!pip install torch-geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
import os, random, numpy as np, torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb
from kaggle_secrets import UserSecretsClient

# W&B login via Kaggle secrets
user_secrets = UserSecretsClient()
wandb.login(key=user_secrets.get_secret("wandb-key"))

EPOCHS = int(os.getenv("EPOCHS", "50"))
NUM_SEEDS = int(os.getenv("NUM_SEEDS", "30"))
PROJECT = "GCN Graph classification(Protein)_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class CentralityEncoder(nn.Module):
    def __init__(self, in_dim=1, out_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 32),
            nn.ReLU(),
            nn.Linear(32, out_dim)
        )
    def forward(self, c):
        return self.net(c)

class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, cent_dim=16):
        super().__init__()
        self.cent_enc = CentralityEncoder(1, cent_dim)
        self.conv1 = GCNConv(in_channels + cent_dim, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, out_channels)
    def forward(self, x, cent, edge_index, batch):
        x = torch.cat([x, self.cent_enc(cent)], dim=1)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

def _minmax_norm(t):
    if len(t) == 0:
        return t
    vmin, vmax = t.min(), t.max()
    if (vmax - vmin) > 0:
        return (t - vmin) / (vmax - vmin)
    return torch.zeros_like(t)

def make_centrality_transform():
    def transform(data):
        if getattr(data, "x", None) is None:
            data.x = torch.ones((data.num_nodes, 1), dtype=torch.float)
        G = to_networkx(data, to_undirected=True)
        cent = torch.zeros((data.num_nodes, 1), dtype=torch.float)
        if G.number_of_nodes() > 0:
            try:
                ev = nx.eigenvector_centrality_numpy(G)
            except Exception:
                ev = {n: 0.0 for n in G.nodes}
            ev_tensor = torch.tensor([ev.get(n, 0.0) for n in range(data.num_nodes)], dtype=torch.float)
            cent[:, 0] = _minmax_norm(ev_tensor)
        data.cent = cent
        return data
    return transform

def run_seed(seed):
    set_seed(seed)
    dataset = TUDataset(root="data/TUDataset", name="PROTEINS", transform=make_centrality_transform())
    dataset = dataset.shuffle()
    train_dataset = dataset[:len(dataset)//2]
    test_dataset = dataset[len(dataset)//2:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = GCN(dataset.num_features if dataset.num_features else 1, 64, dataset.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_Centrality_PROTEINS_seed_{seed}",
        group="GCN_PROTEINS_Centrality",
        job_type="centrality_only",
        reinit=True,
        config={
            "model": "GCN",
            "hidden_channels": 64,
            "epochs": EPOCHS,
            "lr": 0.001,
            "dataset": "PROTEINS",
            "task": "Centrality_eigenvector",
            "seed": seed
        }
    )

    def train():
        model.train()
        total_loss, correct, count = 0.0, 0, 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.cent, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
            count += data.num_graphs
        return total_loss / len(train_loader), correct / count

    def test(loader):
        model.eval()
        correct, count = 0, 0
        for data in loader:
            data = data.to(device)
            out = model(data.x, data.cent, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
            count += data.num_graphs
        return correct / count

    for epoch in range(1, EPOCHS + 1):
        loss, train_acc = train()
        test_acc = test(test_loader)
        print(f"Seed {seed} | Epoch {epoch:03d} | Loss: {loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({
            "epoch": epoch,
            "loss": loss,
            "train_acc": train_acc,
            "test_acc": test_acc
        })

    wandb.finish()

if __name__ == "__main__":
    for s in range(1, NUM_SEEDS + 1):
        run_seed(s)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: galsalman (noynetanel1-tel-aviv-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
Processing...
Done!
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250914_172950-lhypynj0
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run GCN_Centrality_PROTEINS_seed_1
wandb: ⭐️ View project at https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal
wandb: 🚀 View run at https://wandb

Seed 1 | Epoch 001 | Loss: 0.6825 | Train Acc: 0.5665 | Test Acc: 0.5961
Seed 1 | Epoch 002 | Loss: 0.6666 | Train Acc: 0.6007 | Test Acc: 0.5978
Seed 1 | Epoch 003 | Loss: 0.6562 | Train Acc: 0.6025 | Test Acc: 0.6230
Seed 1 | Epoch 004 | Loss: 0.6476 | Train Acc: 0.6367 | Test Acc: 0.6553
Seed 1 | Epoch 005 | Loss: 0.6215 | Train Acc: 0.6906 | Test Acc: 0.6625
Seed 1 | Epoch 006 | Loss: 0.6064 | Train Acc: 0.6960 | Test Acc: 0.6732
Seed 1 | Epoch 007 | Loss: 0.6000 | Train Acc: 0.6978 | Test Acc: 0.6661
Seed 1 | Epoch 008 | Loss: 0.5866 | Train Acc: 0.7122 | Test Acc: 0.6750
Seed 1 | Epoch 009 | Loss: 0.5870 | Train Acc: 0.7194 | Test Acc: 0.6786
Seed 1 | Epoch 010 | Loss: 0.5822 | Train Acc: 0.7086 | Test Acc: 0.6804
Seed 1 | Epoch 011 | Loss: 0.5761 | Train Acc: 0.7122 | Test Acc: 0.6786
Seed 1 | Epoch 012 | Loss: 0.5661 | Train Acc: 0.7176 | Test Acc: 0.6984
Seed 1 | Epoch 013 | Loss: 0.5638 | Train Acc: 0.7302 | Test Acc: 0.6804
Seed 1 | Epoch 014 | Loss: 0.5662 | Train Acc: 0.73

wandb: uploading output.log; uploading wandb-summary.json; uploading config.yaml
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
wandb:      loss ██▆▅▅▄▄▄▃▃▃▃▂▃▂▃▃▂▃▃▂▃▂▂▂▂▂▂▁▁▁▂▁▁▂▁▁▂▁▁
wandb:  test_acc ▁▁▃▅▅▅▆▆▆▆▆▇▇▆▇▇▇▆▆▇▇▇▇█▇▇▇███▇▇█▇▇▇▇█▇▇
wandb: train_acc ▁▂▂▄▆▆▇▇▇▇▇█▇█▇█▇▇▇██▇▇█▇█▇▇███▇▇█▇██▇██
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.51269
wandb:  test_acc 0.69479
wandb: train_acc 0.7446
wandb: 
wandb: 🚀 View run GCN_Centrality_PROTEINS_seed_1 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal/runs/lhypynj0
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250914_172950-lhypynj0/logs
wandb: Tra

Seed 2 | Epoch 001 | Loss: 0.6743 | Train Acc: 0.5683 | Test Acc: 0.5781
Seed 2 | Epoch 002 | Loss: 0.6567 | Train Acc: 0.6151 | Test Acc: 0.5907
Seed 2 | Epoch 003 | Loss: 0.6517 | Train Acc: 0.6529 | Test Acc: 0.5907
Seed 2 | Epoch 004 | Loss: 0.6395 | Train Acc: 0.6439 | Test Acc: 0.6463
Seed 2 | Epoch 005 | Loss: 0.6238 | Train Acc: 0.6799 | Test Acc: 0.6589
Seed 2 | Epoch 006 | Loss: 0.6215 | Train Acc: 0.6799 | Test Acc: 0.6589
Seed 2 | Epoch 007 | Loss: 0.5997 | Train Acc: 0.6906 | Test Acc: 0.6661
Seed 2 | Epoch 008 | Loss: 0.6026 | Train Acc: 0.6960 | Test Acc: 0.6786
Seed 2 | Epoch 009 | Loss: 0.6067 | Train Acc: 0.6960 | Test Acc: 0.6786
Seed 2 | Epoch 010 | Loss: 0.5902 | Train Acc: 0.7140 | Test Acc: 0.6948
Seed 2 | Epoch 011 | Loss: 0.5817 | Train Acc: 0.7158 | Test Acc: 0.6912
Seed 2 | Epoch 012 | Loss: 0.5738 | Train Acc: 0.7248 | Test Acc: 0.6912
Seed 2 | Epoch 013 | Loss: 0.5652 | Train Acc: 0.7302 | Test Acc: 0.6966
Seed 2 | Epoch 014 | Loss: 0.5706 | Train Acc: 0.71

wandb: uploading output.log; uploading wandb-summary.json; uploading config.yaml
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss █▇▇▆▆▅▅▅▄▄▃▃▃▃▃▂▃▂▂▃▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▂▁▁▁
wandb:  test_acc ▁▂▂▄▅▅▆▆▇▇▇▇▇▆▇█▇█▇█▇▇█▇▇███▇▇▇▆██▆▆▇▇█▇
wandb: train_acc ▁▃▄▅▅▆▆▇▇▇▆▇▇▇▇▇▇█▇▇▇█▇▇▇███████▇███████
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.5149
wandb:  test_acc 0.69659
wandb: train_acc 0.7446
wandb: 
wandb: 🚀 View run GCN_Centrality_PROTEINS_seed_2 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal/runs/jn7hr2dq
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250914_173147-jn7hr2dq/logs
wandb: Trac

...

wandb: uploading output.log; uploading config.yaml
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss ██▇▇▆▆▅▅▄▄▄▅▄▃▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▃▁▁
wandb:  test_acc ▁▁▂▂▄▃▄▅▃▅▆▆▆▆▇▆▆▇▇█▇▇█▇██▆███▇███▇▇▇▇█▇
wandb: train_acc ▁▁▂▁▄▅▅▅▅▅▅▆▆▇▇▆▆▇█▇█▇█▇█▇▇██▇█▇███▇▇▇█▇
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.55399
wandb:  test_acc 0.71634
wandb: train_acc 0.72482
wandb: 
wandb: 🚀 View run GCN_Centrality_PROTEINS_seed_28 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal/runs/hi8dlzh1
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250914_182229-hi8dlzh1/logs
wandb: Tracking run with wandb version

Seed 29 | Epoch 001 | Loss: 0.6942 | Train Acc: 0.5216 | Test Acc: 0.6086
Seed 29 | Epoch 002 | Loss: 0.6735 | Train Acc: 0.5827 | Test Acc: 0.6086
Seed 29 | Epoch 003 | Loss: 0.6689 | Train Acc: 0.5827 | Test Acc: 0.6212
Seed 29 | Epoch 004 | Loss: 0.6603 | Train Acc: 0.6493 | Test Acc: 0.6230
Seed 29 | Epoch 005 | Loss: 0.6485 | Train Acc: 0.6385 | Test Acc: 0.6607
Seed 29 | Epoch 006 | Loss: 0.6292 | Train Acc: 0.6888 | Test Acc: 0.6320
Seed 29 | Epoch 007 | Loss: 0.6104 | Train Acc: 0.6727 | Test Acc: 0.6553
Seed 29 | Epoch 008 | Loss: 0.6192 | Train Acc: 0.6601 | Test Acc: 0.6607
Seed 29 | Epoch 009 | Loss: 0.6062 | Train Acc: 0.6799 | Test Acc: 0.6553
Seed 29 | Epoch 010 | Loss: 0.5948 | Train Acc: 0.7140 | Test Acc: 0.6822
Seed 29 | Epoch 011 | Loss: 0.5919 | Train Acc: 0.7230 | Test Acc: 0.6625
Seed 29 | Epoch 012 | Loss: 0.5782 | Train Acc: 0.7140 | Test Acc: 0.6786
Seed 29 | Epoch 013 | Loss: 0.5789 | Train Acc: 0.7086 | Test Acc: 0.6858
Seed 29 | Epoch 014 | Loss: 0.5857 | T

wandb: uploading wandb-summary.json; uploading config.yaml
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss █▇▇▇▆▄▅▄▄▄▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▂▁▁▂▁▁
wandb:  test_acc ▁▁▂▂▄▄▄▄▆▅▆▆▅▇▇▆▇▇▆▆▇▆▆▇▇▇▇█▆▇▇▇▇█▇▇█▇▇▆
wandb: train_acc ▁▃▃▅▄▅▅▆▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇▇██▇██▇███████▇█
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.54237
wandb:  test_acc 0.68223
wandb: train_acc 0.7446
wandb: 
wandb: 🚀 View run GCN_Centrality_PROTEINS_seed_29 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal/runs/ns821std
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250914_182429-ns821std/logs
wandb: Tracking run with wandb 

Seed 30 | Epoch 001 | Loss: 0.6901 | Train Acc: 0.5576 | Test Acc: 0.6014
Seed 30 | Epoch 002 | Loss: 0.6721 | Train Acc: 0.5899 | Test Acc: 0.6014
Seed 30 | Epoch 003 | Loss: 0.6700 | Train Acc: 0.5899 | Test Acc: 0.6014
Seed 30 | Epoch 004 | Loss: 0.6648 | Train Acc: 0.5899 | Test Acc: 0.6014
Seed 30 | Epoch 005 | Loss: 0.6594 | Train Acc: 0.6097 | Test Acc: 0.6158
Seed 30 | Epoch 006 | Loss: 0.6479 | Train Acc: 0.6259 | Test Acc: 0.6535
Seed 30 | Epoch 007 | Loss: 0.6402 | Train Acc: 0.6331 | Test Acc: 0.6715
Seed 30 | Epoch 008 | Loss: 0.6400 | Train Acc: 0.6313 | Test Acc: 0.7127
Seed 30 | Epoch 009 | Loss: 0.6308 | Train Acc: 0.6475 | Test Acc: 0.7056
Seed 30 | Epoch 010 | Loss: 0.6235 | Train Acc: 0.6691 | Test Acc: 0.7127
Seed 30 | Epoch 011 | Loss: 0.6239 | Train Acc: 0.6853 | Test Acc: 0.7092
Seed 30 | Epoch 012 | Loss: 0.6144 | Train Acc: 0.6853 | Test Acc: 0.7092
Seed 30 | Epoch 013 | Loss: 0.6172 | Train Acc: 0.6942 | Test Acc: 0.7145
Seed 30 | Epoch 014 | Loss: 0.6112 | T

wandb: uploading output.log; uploading config.yaml
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
wandb:      loss █▇▇▇▆▅▅▅▅▅▄▄▃▃▃▃▃▂▂▂▂▃▂▂▂▂▂▂▂▁▂▁▂▁▂▁▁▂▂▁
wandb:  test_acc ▁▁▁▁▂▅▇▆▇▇▇▇▇▇█▆▇█▇█▆▇▇▇█▆▆▇▇▇▆▇▇█▇▇▇█▇▇
wandb: train_acc ▁▂▂▂▃▄▄▅▆▆▇▇▇▇█▇▇▇▇▇▇▇█▇██▇█▇██▇▇██▇▇███
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.55915
wandb:  test_acc 0.71634
wandb: train_acc 0.71223
wandb: 
wandb: 🚀 View run GCN_Centrality_PROTEINS_seed_30 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal/runs/tbu77ex5
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250914_182623-tbu77ex5/logs
